# Proyecto Cambio de Divisas

In [83]:
!pip install sqlalchemy-redshift

In [84]:
#Cargando los paquetes
#Requests nos proporciona las capacidades para enviar una solicitud HTTP a un servidor.
import requests
import yaml
import json
import pytz
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine, text
import psycopg2

In [85]:
with open('apikey.yaml', 'r') as file:
    api_keys = yaml.safe_load(file)
api_key = api_keys['api_key']

### Obteniendo el cierre - código que se ejecutará diario

In [86]:
# Extraer la fecha actual para el horario de Mx
timezone = pytz.timezone('America/Mexico_City')
current_time = datetime.now(timezone)
formatted_date = current_time.strftime("%Y-%m-%d")

currencies_list = ['EUR', 'GBP', 'USD', 'PEN', 'BTC', 'KRW', 'INR', 'CNY', 'BRL', 'ARS', 'JPY']
currencies = ','.join(currencies_list)

base_url = f"https://api.apilayer.com/currency_data/timeframe?start_date={formatted_date}&end_date={formatted_date}&source=MXN&currencies={currencies}"

payload = {}
headers= {
  "apikey": api_key
}


response = requests.get(base_url, headers=headers, data=payload)
response.status_code

200

In [87]:
print(json.dumps(response.json(), indent=4))

{
    "success": true,
    "timeframe": true,
    "start_date": "2024-01-21",
    "end_date": "2024-01-21",
    "source": "MXN",
    "quotes": {
        "2024-01-21": {
            "MXNEUR": 0.053512,
            "MXNGBP": 0.045975,
            "MXNUSD": 0.058406,
            "MXNPEN": 0.217969,
            "MXNBTC": 1.399412e-06,
            "MXNKRW": 78.022307,
            "MXNINR": 4.855018,
            "MXNCNY": 0.415683,
            "MXNBRL": 0.288,
            "MXNARS": 47.824295,
            "MXNJPY": 8.654347
        }
    }
}


In [88]:
datos = json.loads(response.text)

Este dataframe trae únicamente una fila, la cual es el valor actual del día en curso que se estará almacenando en la tabla

In [89]:
df = pd.DataFrame(datos)
df

,success,timeframe,start_date,end_date,source,quotes
2024-01-21,True,True,2024-01-21,2024-01-21,MXN,"{'MXNEUR': 0.053512, 'MXNGBP': 0.045975, 'MXNU..."


### Limpiando datos

In [90]:
df_quotes = pd.json_normalize(df['quotes'])


# Combinar las nuevas columnas con el DataFrame original
df = pd.concat([df, df_quotes], axis=1).drop('quotes', axis=1)

df

,success,timeframe,start_date,end_date,source,MXNEUR,MXNGBP,MXNUSD,MXNPEN,MXNBTC,MXNKRW,MXNINR,MXNCNY,MXNBRL,MXNARS,MXNJPY
2024-01-21,True,True,2024-01-21,2024-01-21,MXN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,0.053512,0.045975,0.058406,0.217969,0.000001,78.022307,4.855018,0.415683,0.288,47.824295,8.654347


In [91]:
combined_row = df.iloc[0].combine_first(df.iloc[1])
# Convertir la serie combinada en un DataFrame de una sola fila
df_combined = pd.DataFrame([combined_row])
df_combined.rename(columns=lambda x: x.replace('MXN', '') if 'MXN' in x else x, inplace=True)
df_combined.rename(columns=lambda x: x.replace('start_date', 'date') if 'start_date' in x else x, inplace=True)
columns_to_drop = ['success', 'timeframe','end_date']
df_clean = df_combined.drop(columns=columns_to_drop).reset_index(drop=True)
df_clean

,date,source,EUR,GBP,USD,PEN,BTC,KRW,INR,CNY,BRL,ARS,JPY
0,2024-01-21,MXN,0.053512,0.045975,0.058406,0.217969,0.000001,78.022307,4.855018,0.415683,0.288,47.824295,8.654347


In [92]:
# Conocer el tipo de dato en el df
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    1 non-null      object 
 1   source  1 non-null      object 
 2   EUR     1 non-null      float64
 3   GBP     1 non-null      float64
 4   USD     1 non-null      float64
 5   PEN     1 non-null      float64
 6   BTC     1 non-null      float64
 7   KRW     1 non-null      float64
 8   INR     1 non-null      float64
 9   CNY     1 non-null      float64
 10  BRL     1 non-null      float64
 11  ARS     1 non-null      float64
 12  JPY     1 non-null      float64
dtypes: float64(11), object(2)
memory usage: 232.0+ bytes


### Enviando datos a Redshift

In [93]:
# Cargar credenciales desde el archivo YAML
from sqlalchemy import inspect
with open('credentials.yaml', 'r') as file:
    credenciales = yaml.safe_load(file)['redshift']

# Utilizar credenciales ocultas
username = credenciales['username']
password = credenciales['password']
host = credenciales['host']
port = credenciales['port']
database = credenciales['database']

# Construir la cadena de conexión
cadena_conexion = f"postgresql://{username}:{password}@{host}:{port}/{database}"

# Conectar al motor de la base de datos
engine = create_engine(cadena_conexion)
df_clean.to_sql('cambios_divisa', engine, if_exists='append', index=False)


1

### Leer los datos de redshift que fueron cargador previamente (al día actual) + los que ya se habían cargado (histórico)

In [94]:
# Comprobar que los datos fueron cargados exitosamente en redshift al ser llamados
query = "SELECT * FROM cambios_divisa"
existing_data = pd.read_sql(query, engine)
existing_data

,date,source,eur,gbp,usd,pen,btc,krw,inr,cny,brl,ars,jpy,modified_date
0,2023-02-01,MXN,0.048826,0.043409,0.053770,0.206878,0.000002,65.721149,4.393538,0.362503,0.271805,10.068644,6.916104,2024-01-21 18:58:52.708541
1,2023-02-02,MXN,0.049154,0.043856,0.053601,0.204777,0.000002,65.674506,4.397749,0.360868,0.270722,10.054653,6.901198,2024-01-21 18:58:52.708541
2,2023-02-03,MXN,0.048691,0.043760,0.052721,0.201778,0.000002,65.788509,4.349079,0.357193,0.271648,9.881499,6.916796,2024-01-21 18:58:52.708541
3,2023-02-04,MXN,0.048691,0.043736,0.052721,0.203432,0.000002,65.788509,4.335471,0.357204,0.271611,9.881499,6.916023,2024-01-21 18:58:52.708541
4,2023-02-05,MXN,0.048787,0.043709,0.052644,0.203133,0.000002,65.691804,4.329098,0.356679,0.271211,9.902502,6.949845,2024-01-21 18:58:52.708541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,2024-01-18,MXN,0.053534,0.045823,0.058238,0.216615,0.000001,77.960388,4.843093,0.414943,0.286972,47.710979,8.626918,2024-01-21 18:58:52.708541
352,2024-01-19,MXN,0.053625,0.046072,0.058529,0.219121,0.000001,78.186916,4.865261,0.416560,0.288638,47.903071,8.670581,2024-01-21 18:58:52.708541
353,2024-01-20,MXN,0.053625,0.046072,0.058529,0.218429,0.000001,78.186916,4.865261,0.416560,0.288611,47.827283,8.670581,2024-01-21 18:58:52.708541
354,2024-01-21,MXN,0.053512,0.045975,0.058406,0.217969,0.000001,78.022301,4.855018,0.415683,0.288001,47.824295,8.654351,2024-01-21 18:58:52.708541


### Puede haber duplicados en los datos previos, sin embargo, le pasamos un código para eliminarlos

In [95]:
existing_data.loc[existing_data['modified_date'].isnull(), 'modified_date'] = datetime.now()

# Convertir la columna 'date' a tipo datetime
existing_data['date'] = pd.to_datetime(existing_data['date'])

# Formatear la columna 'date' para mostrar solo la fecha
existing_data['date'] = existing_data['date'].dt.strftime('%Y-%m-%d')

# Ordenar el DataFrame por 'date' de forma descendente y 'modified_date' de forma descendente
existing_data = existing_data.sort_values(by=['date', 'modified_date'], ascending=[False, False])

# Eliminar duplicados basados en la columna 'date' y conservar el primero (el más reciente)
df_combined = existing_data.loc[existing_data.groupby('date')['modified_date'].idxmax()]
df_combined['date'] = pd.to_datetime(df_combined['date'])

df_combined

,date,source,eur,gbp,usd,pen,btc,krw,inr,cny,brl,ars,jpy,modified_date
0,2023-02-01,MXN,0.048826,0.043409,0.053770,0.206878,0.000002,65.721149,4.393538,0.362503,0.271805,10.068644,6.916104,2024-01-21 18:58:52.708541
1,2023-02-02,MXN,0.049154,0.043856,0.053601,0.204777,0.000002,65.674506,4.397749,0.360868,0.270722,10.054653,6.901198,2024-01-21 18:58:52.708541
2,2023-02-03,MXN,0.048691,0.043760,0.052721,0.201778,0.000002,65.788509,4.349079,0.357193,0.271648,9.881499,6.916796,2024-01-21 18:58:52.708541
3,2023-02-04,MXN,0.048691,0.043736,0.052721,0.203432,0.000002,65.788509,4.335471,0.357204,0.271611,9.881499,6.916023,2024-01-21 18:58:52.708541
4,2023-02-05,MXN,0.048787,0.043709,0.052644,0.203133,0.000002,65.691804,4.329098,0.356679,0.271211,9.902502,6.949845,2024-01-21 18:58:52.708541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,2024-01-17,MXN,0.053388,0.045844,0.058114,0.216154,0.000001,78.175645,4.833786,0.414032,0.286821,47.576010,8.607797,2024-01-21 18:58:52.708541
351,2024-01-18,MXN,0.053534,0.045823,0.058238,0.216615,0.000001,77.960388,4.843093,0.414943,0.286972,47.710979,8.626918,2024-01-21 18:58:52.708541
352,2024-01-19,MXN,0.053625,0.046072,0.058529,0.219121,0.000001,78.186916,4.865261,0.416560,0.288638,47.903071,8.670581,2024-01-21 18:58:52.708541
353,2024-01-20,MXN,0.053625,0.046072,0.058529,0.218429,0.000001,78.186916,4.865261,0.416560,0.288611,47.827283,8.670581,2024-01-21 18:58:52.708541


In [96]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 355 entries, 0 to 355
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           355 non-null    datetime64[ns]
 1   source         355 non-null    object        
 2   eur            355 non-null    float64       
 3   gbp            355 non-null    float64       
 4   usd            355 non-null    float64       
 5   pen            355 non-null    float64       
 6   btc            355 non-null    float64       
 7   krw            355 non-null    float64       
 8   inr            355 non-null    float64       
 9   cny            355 non-null    float64       
 10  brl            355 non-null    float64       
 11  ars            355 non-null    float64       
 12  jpy            355 non-null    float64       
 13  modified_date  355 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(11), object(1)
memory usage: 41.6+ KB


### Volver a cargar datos pero ahora evitando duplicados

In [97]:
# Escribir los datos combinados y sin duplicados de nuevo en la base de datos
df_combined['modified_date'] = datetime.now()
df_combined.to_sql('cambios_divisa', engine, if_exists='replace', index=False)
df_combined

,date,source,eur,gbp,usd,pen,btc,krw,inr,cny,brl,ars,jpy,modified_date
0,2023-02-01,MXN,0.048826,0.043409,0.053770,0.206878,0.000002,65.721149,4.393538,0.362503,0.271805,10.068644,6.916104,2024-01-21 18:59:43.927897
1,2023-02-02,MXN,0.049154,0.043856,0.053601,0.204777,0.000002,65.674506,4.397749,0.360868,0.270722,10.054653,6.901198,2024-01-21 18:59:43.927897
2,2023-02-03,MXN,0.048691,0.043760,0.052721,0.201778,0.000002,65.788509,4.349079,0.357193,0.271648,9.881499,6.916796,2024-01-21 18:59:43.927897
3,2023-02-04,MXN,0.048691,0.043736,0.052721,0.203432,0.000002,65.788509,4.335471,0.357204,0.271611,9.881499,6.916023,2024-01-21 18:59:43.927897
4,2023-02-05,MXN,0.048787,0.043709,0.052644,0.203133,0.000002,65.691804,4.329098,0.356679,0.271211,9.902502,6.949845,2024-01-21 18:59:43.927897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,2024-01-17,MXN,0.053388,0.045844,0.058114,0.216154,0.000001,78.175645,4.833786,0.414032,0.286821,47.576010,8.607797,2024-01-21 18:59:43.927897
351,2024-01-18,MXN,0.053534,0.045823,0.058238,0.216615,0.000001,77.960388,4.843093,0.414943,0.286972,47.710979,8.626918,2024-01-21 18:59:43.927897
352,2024-01-19,MXN,0.053625,0.046072,0.058529,0.219121,0.000001,78.186916,4.865261,0.416560,0.288638,47.903071,8.670581,2024-01-21 18:59:43.927897
353,2024-01-20,MXN,0.053625,0.046072,0.058529,0.218429,0.000001,78.186916,4.865261,0.416560,0.288611,47.827283,8.670581,2024-01-21 18:59:43.927897
